In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [2]:
import dgl
import torch
import gurobipy
import numpy as np

from src.problem import get_model, load_instance

instance = load_instance('../data/raw/97_9.jl')
J = instance['jobs'][0]
T = instance['tamanho'][0]

model = get_model(list(range(J)), instance, coupling=True)

model.setParam('TimeLimit', 5)
model.update()
model.optimize()

model.status

/home/bruno/miniconda3/envs/sat-gnn/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-14


9

In [3]:
x = np.random.randint(0, 2, (J,T), dtype=int)

# simulate model output (only the decision variable)
for j in range(J):
    for t in range(T):
        x[j,t] = model.getVarByName(f"x({j},{t})").X

# fresh model
model = get_model(list(range(J)), instance, coupling=True)

# force decision variables to be the same as the incumbent
for j in range(J):
    for t in range(T):
        model.getVarByName(f"x({j},{t})").lb = x[j,t]
        model.getVarByName(f"x({j},{t})").ub = x[j,t]

model.update()
model.optimize()

model.status

2

In [4]:
new_x = np.zeros((J,T), dtype=int)

for j in range(J):
    for t in range(T):
        new_x[j,t] = model.getVarByName(f"x({j},{t})").X

(new_x == x).all()

True